Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

<H3>Logistic regression</H3>

In [18]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Constants.
  beta = tf.constant(0.02)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  regularization = 0.5 * beta * tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + regularization
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [19]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 43.939232
Training accuracy: 13.1%
Validation accuracy: 17.1%
Loss at step 100: 11.945477
Training accuracy: 74.6%
Validation accuracy: 72.5%
Loss at step 200: 4.512330
Training accuracy: 79.6%
Validation accuracy: 76.8%
Loss at step 300: 1.979271
Training accuracy: 82.5%
Validation accuracy: 79.8%
Loss at step 400: 1.118456
Training accuracy: 84.2%
Validation accuracy: 81.3%
Loss at step 500: 0.822643
Training accuracy: 84.9%
Validation accuracy: 82.0%
Loss at step 600: 0.719717
Training accuracy: 85.2%
Validation accuracy: 82.2%
Loss at step 700: 0.683550
Training accuracy: 85.2%
Validation accuracy: 82.3%
Loss at step 800: 0.670735
Training accuracy: 85.3%
Validation accuracy: 82.3%
Test accuracy: 88.8%


<H3>Stochastic gradient descent</H3>

In [36]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Constants.
  beta = tf.constant(0.01)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  regularization = 0.5 * beta * tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [37]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 32.775002
Minibatch accuracy: 7.8%
Validation accuracy: 12.4%
Minibatch loss at step 500: 1.862135
Minibatch accuracy: 83.6%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 0.742987
Minibatch accuracy: 82.0%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 0.679686
Minibatch accuracy: 80.5%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 0.564191
Minibatch accuracy: 85.9%
Validation accuracy: 80.6%
Minibatch loss at step 2500: 0.674817
Minibatch accuracy: 82.8%
Validation accuracy: 81.5%
Minibatch loss at step 3000: 0.643029
Minibatch accuracy: 83.6%
Validation accuracy: 81.2%
Test accuracy: 88.1%


<H3>Neural network</H3>

In [39]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Constants.
  beta = tf.constant(0.01)
  
  # Variables in.
  weights_in = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_in = tf.Variable(tf.zeros([hidden_nodes]))
   
  # Variables out.
  weights_out = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1 = tf.matmul(tf_train_dataset, weights_in) + biases_in
  hidden_layer = tf.nn.relu(layer1)
  logits = tf.matmul(hidden_layer, weights_out) + biases_out
  regularization = 0.5 * beta * (tf.nn.l2_loss(weights_in) + tf.nn.l2_loss(weights_out))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_in) + biases_in), weights_out) + biases_out)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_in) + biases_in), weights_out) + biases_out)

In [40]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1879.867798
Minibatch accuracy: 9.4%
Validation accuracy: 28.6%
Minibatch loss at step 500: 960.969971
Minibatch accuracy: 82.8%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 576.906128
Minibatch accuracy: 85.2%
Validation accuracy: 82.1%
Minibatch loss at step 1500: 350.023682
Minibatch accuracy: 80.5%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 210.444214
Minibatch accuracy: 90.6%
Validation accuracy: 83.1%
Minibatch loss at step 2500: 127.527397
Minibatch accuracy: 89.1%
Validation accuracy: 84.0%
Minibatch loss at step 3000: 77.373756
Minibatch accuracy: 89.1%
Validation accuracy: 85.8%
Test accuracy: 92.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [41]:
batch_size = 2
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Constants.
  beta = tf.constant(0.01)
  
  # Variables in.
  weights_in = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_in = tf.Variable(tf.zeros([hidden_nodes]))
   
  # Variables out.
  weights_out = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1 = tf.matmul(tf_train_dataset, weights_in) + biases_in
  hidden_layer = tf.nn.relu(layer1)
  logits = tf.matmul(hidden_layer, weights_out) + biases_out
  regularization = 0.5 * beta * (tf.nn.l2_loss(weights_in) + tf.nn.l2_loss(weights_out))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_in) + biases_in), weights_out) + biases_out)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_in) + biases_in), weights_out) + biases_out)

In [42]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1898.812500
Minibatch accuracy: 0.0%
Validation accuracy: 13.4%
Minibatch loss at step 500: 1270.552979
Minibatch accuracy: 100.0%
Validation accuracy: 60.7%
Minibatch loss at step 1000: 1085.060547
Minibatch accuracy: 50.0%
Validation accuracy: 62.1%
Minibatch loss at step 1500: 1019.352966
Minibatch accuracy: 100.0%
Validation accuracy: 62.8%
Minibatch loss at step 2000: 866.886536
Minibatch accuracy: 100.0%
Validation accuracy: 55.8%
Minibatch loss at step 2500: 729.106323
Minibatch accuracy: 50.0%
Validation accuracy: 59.4%
Minibatch loss at step 3000: 447.028076
Minibatch accuracy: 100.0%
Validation accuracy: 64.2%
Test accuracy: 69.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [56]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Constants.
  beta = tf.constant(0.01)
  
  # Variables in.
  weights_in = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_in = tf.Variable(tf.zeros([hidden_nodes]))
   
  # Variables out.
  weights_out = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  keep_prob = tf.placeholder(tf.float32)
  layer1 = tf.matmul(tf_train_dataset, weights_in) + biases_in
  hidden_layer = tf.nn.dropout(tf.nn.relu(layer1), 0.75)
  logits = tf.matmul(hidden_layer, weights_out) + biases_out
  regularization = 0.5 * beta * (tf.nn.l2_loss(weights_in) + tf.nn.l2_loss(weights_out))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_in) + biases_in), weights_out) + biases_out)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_in) + biases_in), weights_out) + biases_out)

In [57]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1947.385498
Minibatch accuracy: 9.4%
Validation accuracy: 28.3%
Minibatch loss at step 500: 957.078674
Minibatch accuracy: 80.5%
Validation accuracy: 82.8%
Minibatch loss at step 1000: 573.791626
Minibatch accuracy: 80.5%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 346.914276
Minibatch accuracy: 80.5%
Validation accuracy: 84.0%
Minibatch loss at step 2000: 209.275360
Minibatch accuracy: 83.6%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 126.591843
Minibatch accuracy: 82.8%
Validation accuracy: 84.6%
Minibatch loss at step 3000: 76.901817
Minibatch accuracy: 85.9%
Validation accuracy: 86.3%
Test accuracy: 92.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [82]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Constants.
  beta = 0.00001
  global_step = tf.Variable(0)  # count the number of steps taken.
  dropout_keeprate = 0.5
  learning_rate = 0.75
  decay_rate = 0.98
  decay_steps = 1000
  
  # Variables in.
  weights_in = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_in = tf.Variable(tf.zeros([hidden_nodes]))
   
  # Variables out.
  weights_out = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  keep_prob = tf.placeholder(tf.float32)
  layer1 = tf.matmul(tf_train_dataset, weights_in) + biases_in
  hidden_layer = tf.nn.dropout(tf.nn.relu(layer1), dropout_keeprate)
  logits = tf.matmul(hidden_layer, weights_out) + biases_out
  regularization = 0.5 * beta * (tf.nn.l2_loss(weights_in) + tf.nn.l2_loss(weights_out))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + regularization
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(learning_rate, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_in) + biases_in), weights_out) + biases_out)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_in) + biases_in), weights_out) + biases_out)

In [83]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 461.536102
Minibatch accuracy: 8.6%
Validation accuracy: 42.0%
Minibatch loss at step 500: 36.497135
Minibatch accuracy: 69.5%
Validation accuracy: 78.0%
Minibatch loss at step 1000: 12.347488
Minibatch accuracy: 73.4%
Validation accuracy: 76.0%
Minibatch loss at step 1500: 21.622919
Minibatch accuracy: 64.8%
Validation accuracy: 75.5%
Minibatch loss at step 2000: 14.368712
Minibatch accuracy: 75.8%
Validation accuracy: 76.0%
Minibatch loss at step 2500: 15.676882
Minibatch accuracy: 68.0%
Validation accuracy: 75.5%
Minibatch loss at step 3000: 11.213916
Minibatch accuracy: 75.0%
Validation accuracy: 75.8%
Minibatch loss at step 3500: 8.363549
Minibatch accuracy: 69.5%
Validation accuracy: 77.2%
Minibatch loss at step 4000: 4.621014
Minibatch accuracy: 74.2%
Validation accuracy: 78.2%
Minibatch loss at step 4500: 6.257516
Minibatch accuracy: 77.3%
Validation accuracy: 79.9%
Minibatch loss at step 5000: 3.257499
Minibatch accuracy: 77.3%
Validation 